## Instalacja i Import

In [28]:
import sys
import subprocess

print(f"Python executable: {sys.executable}")
print("Instaluję Groq SDK...")

try:
    result = subprocess.run(
        [sys.executable, "-m", "pip", "install", "--user", "groq", "scikit-learn"],
        capture_output=True,
        text=True,
        check=True
    )
    print("[OK] Instalacja zakończona!")
    print(result.stdout)
except subprocess.CalledProcessError as e:
    print(f"[ERROR] Błąd instalacji:")
    print(e.stderr)

try:
    from sklearn.metrics import accuracy_score
    from groq import Groq
    import sklearn
    print(f"\n[OK] sklearn {sklearn.__version__} oraz Groq SDK zainstalowane!")
except ImportError as e:
    print(f"\n[ERROR] Import failed: {e}")

Python executable: c:\Users\olobr\Lupa-na-prompt\.venv\Scripts\python.exe
Instaluję Groq SDK...
[ERROR] Błąd instalacji:
ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.


[OK] sklearn 1.8.0 oraz Groq SDK zainstalowane!


## Wczytanie danych

In [29]:
import pandas as pd
import numpy as np

df = pd.read_csv('prompts2.csv', sep=';')
print(f"Shape: {df.shape}")
print(f"Prompt dtype: {df['Prompt'].dtype}, Flag dtype: {df['Flag'].dtype}")
print(f"\nPierwsze 3 wiersze:")
print(df.head(3))

Shape: (60, 2)
Prompt dtype: object, Flag dtype: int64

Pierwsze 3 wiersze:
                                              Prompt  Flag
0  Przecięcie dowolnej, nawet nieprzeliczalnej ro...     1
1  Skończona addytywność prawdopodobieństwa jest ...     0
2  Jeśli ciąg zdarzeń jest wstępujący, to prawdop...     1


## Wczytanie kontekstu (opcjonalne)

In [30]:
import pathlib
kontekst = "ROZPOCZĘCIE WYKŁADU METODY NUMERYCZNE \n"
kontekst += pathlib.Path('context/MN.md').read_text(encoding='utf-8')
kontekst += "ROZPOCZĘCIE WYKŁADU RACHUNEK PRAWDOBIEŃSTWA \n"
kontekst += pathlib.Path('context/prob.md').read_text(encoding='utf-8')

print(f"Kontekst załadowany: {len(kontekst)} znaków")

Kontekst załadowany: 326979 znaków


## Groq API Setup

In [31]:
from groq import Groq

groq_api_key = "GROQ_API_KEY_PLACEHOLDER"
client = Groq(api_key=groq_api_key)

# Test połączenia
print("Testing Groq API connection...")
test_response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {"role": "user", "content": "Czy działasz? Odpowiedz jednym słowem."}
    ],
    temperature=0.5,
    max_tokens=50
)
print(f"[OK] Model llama-3.3-70b-versatile działa!")
print(f"Odpowiedź: {test_response.choices[0].message.content}")

Testing Groq API connection...
[OK] Model llama-3.3-70b-versatile działa!
Odpowiedź: Tak.


## DataModel dla Groq

In [32]:
import time
import asyncio
import string
from random import sample, choice
from tqdm import tqdm

class DataModelGroq:
    def __init__(self, df, client, rate_limit=30, positive_response="'1'", negative_response="'0'"):
        self.df = df.copy()
        self.responses = pd.DataFrame(index=df.index)
        self.responses_parsed = pd.DataFrame(index=df.index)
        self.client = client
        self.columns_with_prompts = ['Prompt']
        self.rate_limit = rate_limit
        self.positive_response = positive_response
        self.negative_response = negative_response
        self.stats = {'OK': 0, 'ERR': 0}

    def __call__(self):
        print(f"Typ kolumny Prompt: {type(self.df['Prompt'])}")
        print(f"Wymiary danych: {self.df.shape}")
        print(f"Dostępne kolumny: {self.df.columns.to_list()}")
        print(f"Kolumny z promptami do przetworzenia: {self.columns_with_prompts}")

    def make_prompts(self):
        # Modyfikuj prompt aby wymuszić odpowiedź 1 lub 0
        self.df["Prompt"] = self.df["Prompt"].apply(
            lambda x: f"{x}\nOdpowiedz TYLKO '1' jeśli zdanie jest PRAWDZIWE albo '0' jeśli FAŁSZYWE. BEZ ŻADNYCH DODATKOWYCH SŁÓW."
        )
        
        def inject_noise(text, rate=0.1):
            if not isinstance(text, str): return str(text)
            indices = [i for i, c in enumerate(text) if c.isalpha()]
            if not indices: return text
            k = int(len(indices) * rate)
            text_list = list(text)
            for i in sample(indices, k):
                text_list[i] = choice(string.ascii_letters)
            return "".join(text_list)

        strategies = {
            'Negative_prompt': lambda x: f"Myślę, że to zdanie zawiera problem, czy mam rację?: {x}",
            'Positive_prompt': lambda x: f"Myślę, że to zdanie jest poprawne, czy mam rację?: {x}",
            'Positive_Extra_role': lambda x: f"Jesteś ekspertem w dziedzinie rachunku prawdopodobieństwa oraz metod numerycznych, sprawdź prawdziwość: {x}",
            'Negative_Extra_role': lambda x: f"Jesteś ekspertem w dziedzinie socjologii oraz obcych form życia, sprawdź prawdziwość: {x}",
            'Uncertainty_prompt': lambda x: f"Oceń prawdziwość tego zdania tylko jeżeli jesteś pewny odpowiedzi: {x}",
            'Scrambled_prompt': lambda x: f"Oceń prawdziwość (tekst pomieszany): {''.join(np.random.permutation(list(x)))}",
            'Chain_of_thoughts': lambda x: f"Przeanalizuj dokładnie to zdanie krok po kroku i oceń jego prawdziwość: {x}",
            'Sceptical_role': lambda x: f"Jesteś sceptykiem, który zawsze podważa prawdziwość informacji. Oceń: {x}",
            'High_stakes': lambda x: f"Oceń prawdziwość, pamiętając, że od twojej odpowiedzi zależy życie wielu osób: {x}",
            'Tipping': lambda x: f"Dostaniesz duży napiwek jeśli prawidłowo ocenisz to zdanie: {x}",
            'Random_mistake': lambda x: f"Oceń prawdziwość: {inject_noise(x)}"
        }

        for col_name, func in strategies.items():
            self.df[col_name] = self.df["Prompt"].apply(func)
            self.columns_with_prompts.append(col_name)

    def _call_api_with_retry(self, prompt, max_retries=3, base_delay=2):
        """Wywołanie API z retry"""
        for attempt in range(max_retries):
            try:
                response = self.client.chat.completions.create(
                    model="llama-3.3-70b-versatile",
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0.5,
                    max_tokens=20
                )
                return response.choices[0].message.content.strip()
            except Exception as e:
                if attempt == max_retries - 1:
                    return f"Error: {str(e)}"
                time.sleep(base_delay * (2 ** attempt))

    def prompts(self):
        """Generuj odpowiedzi dla wszystkich strategii"""
        total_columns = len(self.columns_with_prompts)
        print(f"Rozpoczynam generowanie dla {total_columns} kolumn...")
        print(f"Rate limit: {self.rate_limit} req/min (delay: {60.0/self.rate_limit:.1f}s)")
        print()
        
        delay_seconds = 60.0 / self.rate_limit
        total_ok = 0
        total_err = 0
        
        for col_index, col_name in enumerate(self.columns_with_prompts):
            print(f"\n--> [{col_index+1}/{total_columns}] Przetwarzanie kolumny: {col_name}")
            prompts_list = self.df[col_name].tolist()
            column_results = []
            
            pbar = tqdm(prompts_list, desc=f"Generating '{col_name}'", unit="prompt")
            
            for prompt in pbar:
                loop_start_time = time.time()
                
                result = self._call_api_with_retry(prompt)
                
                if result.startswith("Error:"):
                    total_err += 1
                else:
                    total_ok += 1
                
                column_results.append(result)
                pbar.set_postfix({'OK': total_ok, 'ERR': total_err})
                
                elapsed = time.time() - loop_start_time
                wait_time = max(0, delay_seconds - elapsed)
                
                if wait_time > 0:
                    time.sleep(wait_time)
            
            self.responses[col_name] = column_results
        
        print(f"\n✓ Zakończono generowanie wszystkich odpowiedzi.")
        print("-" * 50)
        print(f"Sukcesy: {total_ok}")
        print(f"Błędy:   {total_err}")
        print("-" * 50)
        return self.responses

    def parsowanie(self):
        """Parsowanie odpowiedzi do formatu binarnego"""
        if self.responses.empty:
            print("Brak odpowiedzi do sparsowania.")
            return self.responses_parsed

        print(f"Rozpoczynam parsowanie {len(self.responses.columns)} kolumn...")

        def parse_single_response(text):
            if not isinstance(text, str):
                return None
            # Szukaj '1' lub 1
            if '1' in text and ('0' not in text or text.index('1') < text.index('0')):
                return 1
            elif '0' in text:
                return 0
            else:
                return None

        for col_name in self.responses.columns:
            self.responses_parsed[col_name] = self.responses[col_name].apply(parse_single_response)
            valid_count = self.responses_parsed[col_name].notna().sum()
            print(f"--> Kolumna '{col_name}': {valid_count}/{len(self.responses_parsed)}")

        print("✓ Parsowanie zakończone.")
        return self.responses_parsed

print("✓ DataModelGroq loaded")

✓ DataModelGroq loaded


## Krok 1: Przygotowanie promptów (20 first only)

UWAGA: Testujemy tylko na 20 promptach zamiast 60 aby oszczędzić czas

In [33]:
# Weź tylko 20 pierwszych promptów
df_subset = df.iloc[:20].copy()
print(f"Używam {len(df_subset)} promptów (podzbiór)")
print(f"True/False split: {df_subset['Flag'].value_counts().to_dict()}")

data_model_groq = DataModelGroq(df_subset, client, rate_limit=30)
data_model_groq.make_prompts()
data_model_groq()

Używam 20 promptów (podzbiór)
True/False split: {1: 11, 0: 9}
Typ kolumny Prompt: <class 'pandas.core.series.Series'>
Wymiary danych: (20, 13)
Dostępne kolumny: ['Prompt', 'Flag', 'Negative_prompt', 'Positive_prompt', 'Positive_Extra_role', 'Negative_Extra_role', 'Uncertainty_prompt', 'Scrambled_prompt', 'Chain_of_thoughts', 'Sceptical_role', 'High_stakes', 'Tipping', 'Random_mistake']
Kolumny z promptami do przetworzenia: ['Prompt', 'Negative_prompt', 'Positive_prompt', 'Positive_Extra_role', 'Negative_Extra_role', 'Uncertainty_prompt', 'Scrambled_prompt', 'Chain_of_thoughts', 'Sceptical_role', 'High_stakes', 'Tipping', 'Random_mistake']


## Krok 2: GENEROWANIE ODPOWIEDZI Z GROQ API

⚠️ **AKTYWNY KOD** - będzie wywoływać API

In [34]:
# ==============================================
# ZAKOMENTOWANE - NIE URUCHAMIAJ BEZ POTRZEBY!
# Odkomentuj TYLKO jeśli chcesz nowe dane z API
# ==============================================

# responses_df = data_model_groq.prompts()
# parsed_df = data_model_groq.parsowanie()

print("Generowanie ZAKOMENTOWANE")
print("Uzyj ponizszej komorki aby wczytac wyniki z dysku")

Generowanie ZAKOMENTOWANE
Uzyj ponizszej komorki aby wczytac wyniki z dysku


## Wczytanie wyników z dysku

In [35]:
import os
from glob import glob
from pathlib import Path

def load_groq_results(folder='saved_responses_groq'):
    """Wczytuje ostatnie wyniki Groq z dysku"""
    if not os.path.exists(folder):
        print(f'BLAD: Folder {folder} nie istnieje')
        return None, None
    
    parsed_files = glob(f'{folder}/parsed_responses_*.csv')
    if not parsed_files:
        print(f'BLAD: Brak plikow w {folder}')
        return None, None
    
    latest = max(parsed_files, key=os.path.getmtime)
    timestamp = Path(latest).stem.replace('parsed_responses_', '')
    
    parsed_path = f'{folder}/parsed_responses_{timestamp}.csv'
    raw_path = f'{folder}/raw_responses_{timestamp}.csv'
    
    parsed_df = pd.read_csv(parsed_path, index_col=0)
    responses_df = pd.read_csv(raw_path, index_col=0) if os.path.exists(raw_path) else None
    
    print(f'Wczytano: {Path(latest).name}')
    print(f'Shape: {parsed_df.shape}')
    return parsed_df, responses_df

# WCZYTAJ WYNIKI
parsed_df, responses_df = load_groq_results()
print(f'\nGotowe do analizy!')

Wczytano: parsed_responses_2025-12-31_00-25-21.csv
Shape: (20, 12)

Gotowe do analizy!


## Krok 5: Statystyki

In [36]:
# Porównanie z expected labels
expected_labels = df_subset['Flag'].values  # True=1, False=0

print("\n📊 STATYSTYKI NA POZIOMIE KOLUMN (strategie)")
print("="*80)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

metrics_list = []

for col in parsed_df.columns:
    y_pred = parsed_df[col].values
    
    # Sprawdź czy są NaN
    valid_mask = ~pd.isna(y_pred)
    if valid_mask.sum() == 0:
        continue
    
    y_pred_valid = y_pred[valid_mask]
    y_true_valid = expected_labels[valid_mask]
    
    acc = accuracy_score(y_true_valid, y_pred_valid)
    prec = precision_score(y_true_valid, y_pred_valid, zero_division=0)
    rec = recall_score(y_true_valid, y_pred_valid, zero_division=0)
    f1 = f1_score(y_true_valid, y_pred_valid, zero_division=0)
    
    metrics_list.append({
        'Strategy': col,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1': f1,
        'Valid': valid_mask.sum()
    })
    
    print(f"{col:25} | Acc: {acc:.3f} | Prec: {prec:.3f} | Rec: {rec:.3f} | F1: {f1:.3f} | Valid: {valid_mask.sum()}/{len(y_pred)}")

metrics_df = pd.DataFrame(metrics_list)
print("\n" + "="*80)
print(f"ŚREDNIA ACCURACY: {metrics_df['Accuracy'].mean():.3f}")
print(f"Best: {metrics_df.loc[metrics_df['Accuracy'].idxmax(), 'Strategy']} ({metrics_df['Accuracy'].max():.3f})")
print(f"Worst: {metrics_df.loc[metrics_df['Accuracy'].idxmin(), 'Strategy']} ({metrics_df['Accuracy'].min():.3f})")


📊 STATYSTYKI NA POZIOMIE KOLUMN (strategie)
Prompt                    | Acc: 0.650 | Prec: 0.625 | Rec: 0.909 | F1: 0.741 | Valid: 20/20
Negative_prompt           | Acc: 0.700 | Prec: 0.727 | Rec: 0.727 | F1: 0.727 | Valid: 20/20
Positive_prompt           | Acc: 0.700 | Prec: 0.692 | Rec: 0.818 | F1: 0.750 | Valid: 20/20


Positive_Extra_role       | Acc: 0.700 | Prec: 0.692 | Rec: 0.818 | F1: 0.750 | Valid: 20/20
Negative_Extra_role       | Acc: 0.600 | Prec: 0.588 | Rec: 0.909 | F1: 0.714 | Valid: 20/20
Uncertainty_prompt        | Acc: 0.700 | Prec: 0.692 | Rec: 0.818 | F1: 0.750 | Valid: 20/20
Scrambled_prompt          | Acc: 0.000 | Prec: 0.000 | Rec: 0.000 | F1: 0.000 | Valid: 1/20
Chain_of_thoughts         | Acc: 0.750 | Prec: 0.714 | Rec: 0.909 | F1: 0.800 | Valid: 20/20
Sceptical_role            | Acc: 0.700 | Prec: 0.667 | Rec: 0.909 | F1: 0.769 | Valid: 20/20
High_stakes               | Acc: 0.650 | Prec: 0.625 | Rec: 0.909 | F1: 0.741 | Valid: 20/20
Tipping                   | Acc: 0.600 | Prec: 0.600 | Rec: 0.818 | F1: 0.692 | Valid: 20/20
Random_mistake            | Acc: 0.550 | Prec: 0.562 | Rec: 0.818 | F1: 0.667 | Valid: 20/20

ŚREDNIA ACCURACY: 0.608
Best: Chain_of_thoughts (0.750)
Worst: Scrambled_prompt (0.000)


## Analiza wzorcow

In [37]:
baseline_acc = metrics_df[metrics_df['Strategy'] == 'Prompt']['Accuracy'].values[0]
best_acc = metrics_df['Accuracy'].max()
best_strat = metrics_df.loc[metrics_df['Accuracy'].idxmax(), 'Strategy']
worst_acc = metrics_df['Accuracy'].min()
worst_strat = metrics_df.loc[metrics_df['Accuracy'].idxmin(), 'Strategy']

print("=" * 80)
print("ANALIZA WZORCOW - GROQ LLaMA 3.3 70B")
print("=" * 80)

print(f"\nBaseline (Prompt):        {baseline_acc:.1%}")
print(f"Najlepsza ({best_strat:20s}): {best_acc:.1%}  (+{(best_acc-baseline_acc)*100:.1f} pp)")
print(f"Najslabsza ({worst_strat:20s}): {worst_acc:.1%}  ({(worst_acc-baseline_acc)*100:+.1f} pp)")

print("\n" + "=" * 80)
print("WZORCE:")
print("=" * 80)

role_expert = metrics_df[metrics_df['Strategy'] == 'Positive_Extra_role']['Accuracy'].values
role_amateur = metrics_df[metrics_df['Strategy'] == 'Negative_Extra_role']['Accuracy'].values
if len(role_expert) > 0 and len(role_amateur) > 0:
    print(f"* Rola eksperta:     {role_expert[0]:.1%}")
    print(f"* Rola amatora:      {role_amateur[0]:.1%}")
    print(f"  -> roznica = {(role_expert[0] - role_amateur[0])*100:+.1f} pp")

pos = metrics_df[metrics_df['Strategy'] == 'Positive_prompt']['Accuracy'].values
neg = metrics_df[metrics_df['Strategy'] == 'Negative_prompt']['Accuracy'].values
if len(pos) > 0 and len(neg) > 0:
    print(f"\n* Positive framing:  {pos[0]:.1%}")
    print(f"* Negative framing:  {neg[0]:.1%}")
    print(f"  -> roznica = {(pos[0] - neg[0])*100:+.1f} pp")

cot = metrics_df[metrics_df['Strategy'] == 'Chain_of_thoughts']['Accuracy'].values
if len(cot) > 0:
    print(f"\n* Chain-of-thought:  {cot[0]:.1%}")
    print(f"  -> vs baseline: {(cot[0] - baseline_acc)*100:+.1f} pp")

scrambled = metrics_df[metrics_df['Strategy'] == 'Scrambled_prompt']['Accuracy'].values
if len(scrambled) > 0:
    print(f"\n* Scrambled:         {scrambled[0]:.1%}")
    print(f"  -> vs baseline: {(scrambled[0] - baseline_acc)*100:+.1f} pp")

print("\n" + "=" * 80)
print(f"REKOMENDACJA: stosuj strategie '{best_strat}'")
print("=" * 80)

ANALIZA WZORCOW - GROQ LLaMA 3.3 70B

Baseline (Prompt):        65.0%
Najlepsza (Chain_of_thoughts   ): 75.0%  (+10.0 pp)
Najslabsza (Scrambled_prompt    ): 0.0%  (-65.0 pp)

WZORCE:
* Rola eksperta:     70.0%
* Rola amatora:      60.0%
  -> roznica = +10.0 pp

* Positive framing:  70.0%
* Negative framing:  70.0%
  -> roznica = +0.0 pp

* Chain-of-thought:  75.0%
  -> vs baseline: +10.0 pp

* Scrambled:         0.0%
  -> vs baseline: -65.0 pp

REKOMENDACJA: stosuj strategie 'Chain_of_thoughts'
